In [11]:
from keras import backend as K
K.clear_session()

In [12]:
K.set_floatx('float16')

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD
import numpy as np
import csv
import keras

from os import listdir
from os.path import isfile, join


#firstDataSetPath = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/086e42f6-2077-4d8c-a713-6eddd58a4177_TrainingSet_1_of_2/TrainingSet_1_of_2"
#secondDataSetPath = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/07224578-0898-497a-b21d-94fbf883b974_TrainingSet_2_of_2/TrainingSet_2_of_2"

resampleFolder = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImages"

In [5]:
from keras.layers import Conv3D, MaxPool3D, GlobalMaxPool3D

In [13]:
model = Sequential()
model.add(Conv3D(10, 5, input_shape=(512, 512, 130, 1)))
model.add(MaxPool3D(2))
model.add(Conv3D(256, 5))
model.add(MaxPool3D(2))
model.add(Conv3D(128, 3))
model.add(MaxPool3D(2))
model.add(Conv3D(128, 3))
model.add(MaxPool3D(2))
model.add(Conv3D(64, 3))
model.add(MaxPool3D(2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,#'rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 508, 508, 126, 10) 1260      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 254, 254, 63, 10)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 250, 250, 59, 512) 640512    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 125, 125, 29, 512) 0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 123, 123, 27, 256) 3539200   
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 61, 61, 13, 256)   0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 59, 59, 11, 256)   1769728   
__________

In [7]:
x_train = np.zeros((20, 512, 512, 130, 1))
y_train = np.zeros((20, 5))
patientIdentifier = ''
contor = 0

for f in listdir(resampleFolder):
    if(contor < 20):
        if isfile(join(resampleFolder, f)):
            auxImage = np.load(resampleFolder + '/' + f)
            x_train[contor] = np.expand_dims(auxImage, -1)


            patientIdentifier = f.replace('.npy', '')
            with open('/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/8ba42c65-534a-4cdf-b983-e7fcaaaa3002_TrainingSet_metaData.csv', 'r') as csvfile:
                spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
                for row in spamreader:
                    if row[0].split(',')[0] == patientIdentifier:
                        y_train[contor] = keras.utils.to_categorical(int(row[0].split(',')[11]) -1, num_classes=5)
        contor += 1
    else:
        break
            

#print(x_train)

In [8]:
print(x_train.shape)
print(y_train.shape)
print(y_train)

(20, 512, 512, 130, 1)
(20, 5)
[[0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [10]:
K.floatx()

'float16'

In [14]:
model.fit(x_train, y_train,
          epochs=20,
          batch_size=2)

scores = model.evaluate(x_train, y_train, batch_size=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
 



Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[2,250,250,59,512] and type half on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv3d_2/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:

 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model.evaluate(x_test, y_test, batch_size=32)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))